# RooTrip 추천 시스템 - V4
### - 사용자가 해시태그별 좋아요를 누른 숫자를 바탕으로 가중

In [79]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# 데이터프레임 생성
data = {
    'id': [1, 2, 3, 4],
    'Hashtag': [
        [{"tag": "어트랙션"}, {"tag": "여행"}],
        [{"tag": "바다"}, {"tag": "여행"}],
        [{"tag": "산"}, {"tag": "바다"}, {"tag": "관광"}],
        [{"tag": "부산"}, {"tag": "바다"}, {"tag": "해운대"}]
    ],
    'board_id': [1, 2, 3, 4],
    'title': ['어트랙션 여행', '바다 여행', '산 바다 관광', '부산 바다 해운대']
}

df1 = pd.DataFrame(data)

user_likes = [1, 10, 1, 100, 20, 1, 0] # 어트랙션, 여행, 바다, 산, 관광, 해운대, 부산

tag_mapping = {
    "어트랙션": 0,
    "여행": 1,
    "바다": 2,
    "산": 3,
    "관광": 4,
    "해운대": 5,
    "부산": 6
}

def calculate_weights(tags, user_likes):
    weights = [user_likes[tag_mapping.get(tag['tag'], 0)] for tag in tags]
    return ' '.join([str(weight) for weight in weights])

# 각 게시물에 대해 가중치를 계산하여 'weights' 열에 저장
df1['weights'] = df1['Hashtag'].apply(calculate_weights, user_likes=user_likes)
print(df1)

# CountVectorizer를 사용하여 단어 벡터화
count = CountVectorizer()
count_matrix = count.fit_transform(df1['weights'])

# 코사인 유사도 매트릭스 계산
cosine_sim = cosine_similarity(count_matrix, count_matrix)

# 사용자 입력 가중치와 각 행의 가중치 간의 유사도 계산
user_weight_vector = count.transform([' '.join([str(user_likes[i]) for i in range(len(user_likes))])])

similarities = cosine_similarity(user_weight_vector, count_matrix)

# 유사도와 게시글 제목을 매핑
similarities_with_titles = list(enumerate(similarities[0]))
similarities_with_titles = sorted(similarities_with_titles, key=lambda x: x[1], reverse=True)

# 가장 유사한 게시글 추출
recommended_posts = []
for i in range(4):  # 상위 3개 게시글 추천
    post_idx = similarities_with_titles[i][0]
    recommended_posts.append(df1['title'].iloc[post_idx])
    recommended_posts.append(df1['board_id'].iloc[post_idx])

# 사용자에게 추천 목록을 출력
print("추천 게시글:")
for post in recommended_posts:
    print(post)


   id                                         Hashtag  board_id      title  \
0   1                [{'tag': '어트랙션'}, {'tag': '여행'}]         1    어트랙션 여행   
1   2                  [{'tag': '바다'}, {'tag': '여행'}]         2      바다 여행   
2   3    [{'tag': '산'}, {'tag': '바다'}, {'tag': '관광'}]         3    산 바다 관광   
3   4  [{'tag': '부산'}, {'tag': '바다'}, {'tag': '해운대'}]         4  부산 바다 해운대   

    weights  
0      1 10  
1      1 10  
2  100 1 20  
3     0 1 1  
추천 게시글:
산 바다 관광
3
어트랙션 여행
1
바다 여행
2
부산 바다 해운대
4
